# Notebook [3]: Training the reader on the SQuAD v1.1 dataset

This notebook shows how to fine-tune a pre-trained BERT model on the SQuAD.

***Note:*** *To run this notebook you will need to have access to GPU. The fine-tuning of the Reader was done with an AWS EC2 p3.2xlarge machine (GPU Tesla V100 16GB). It took about 2 hours to complete (2 epochs on SQuAD 1.1 train was enough to achieve SOTA results on SQuAD 1.1 dev).*

In [ ]:
import os
import torch
from sklearn.externals import joblib
from cdqa.reader.bertqa_sklearn import BertProcessor, BertQA

### Preprocess SQuAD examples

In [ ]:
train_processor = BertProcessor(do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X='data/train-v1.1.json')

### Train the model

In [ ]:
reader = BertQA(train_batch_size=12,
                learning_rate=3e-5,
                num_train_epochs=2,
                do_lower_case=True,
                output_dir='models')

reader.fit(X=(train_examples, train_features))

### Save GPU model locally

In [ ]:
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa_vGPU.joblib'))

### Send model to CPU

In [ ]:
reader.model.to('cpu')
reader.device = torch.device('cpu')

### Save CPU model locally

In [ ]:
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa_vCPU.joblib'))